In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

def load_data(file_path, data, split):

    return pd.read_pickle(file_path + data + "_" + split + ".pkl")

def create_data(Type='PdM2', split='Train', save=True):

    """
        Custom dataset must include:
            age: time since last failure
            Failure: 1 = failed, 0 = did not fail
            ttf: time to next failure (For display purposes, would not be fed into the model)
            Date: must be sorted sequentially
        
        Splits: Train, Test, None
    """

    file_path = 'Gym-PM/gym_pm/data/' + Type + '.csv'
    df = pd.read_csv(file_path)

    if Type == 'PdM2':

        df['Failure'] = df.Failure_today.apply(lambda x: 0 if x == 'No' else 1)
        df.Date = pd.to_datetime(df.Date)

        cutoff = '2016-10'
        if split == 'Train':
            df = df[(df.Date > '2015-03') & (df.Date < cutoff)]
        elif split == 'Test':
            df = df[df.Date >= cutoff]
        elif split == 'All':
            df = df[df.Date > '2015-03']
        else:
            return "Invalid Split"

        df = df.sort_values('Date')
        df.drop(columns = ['Fail_tomorrow', 'Failure_today', 'Parameter1_Dir', 
                           'Parameter2_9am', 'Parameter2_3pm'], inplace = True)
        df.fillna(0, inplace = True)
        df.reset_index(drop = True, inplace = True)

        # ttf
        failure_time = df.groupby(['Location', 'Date']).Failure.first() 
        failure_time = failure_time[failure_time == 1].reset_index() # Collect failure dates
        failure_time = failure_time.groupby('Location').Date.apply(np.array)
        failure_time = failure_time.reset_index()

        failure_time.rename(columns = {"Date": "ttf"}, inplace = True)
        df = df.merge(failure_time, how = 'inner', on = 'Location')
        df['age'] = df.ttf

        df.ttf = df.apply(lambda x: x['ttf'][x['ttf'] >= x['Date']], axis = 1)
        df = df[df.ttf.str.len() > 0] # Drop empty lists
        df.ttf = df.apply(lambda x: (x['ttf'][0] - x['Date']).days, axis = 1) # Calculate TTF

        # age
        df.age = df.apply(lambda x: x['age'][x['age'] < x['Date']], axis = 1)
        df = df[df.age.str.len() > 0] # Drop empty lists
        df.age = df.apply(lambda x: (x['Date'] - x['age'][-1]).days, axis = 1) # Calculate Age
        df = df[~(df.age + df.ttf <= 10)]
        df.reset_index(drop = True, inplace = True)
        df.drop(columns = ['Date', 'Location'], inplace = True)

        # Save data as pickle
        if save:
            df.to_pickle(file_path.replace('.csv', '_' + split + '.pkl'))

    return df

df = create_data('PdM2', split='Train', save=True)
df.columns

Index(['Min_Temp', 'Max_Temp', 'Leakage', 'Evaporation', 'Electricity',
       'Parameter1_Speed', 'Parameter3_9am', 'Parameter3_3pm',
       'Parameter4_9am', 'Parameter4_3pm', 'Parameter5_9am', 'Parameter5_3pm',
       'Parameter6_9am', 'Parameter6_3pm', 'Parameter7_9am', 'Parameter7_3pm',
       'RISK_MM', 'Failure', 'ttf', 'age'],
      dtype='object')

In [2]:
# Import Modules
import numpy as np
from timeseries_generator.external_factors import CountryGdpFactor, EUIndustryProductFactor
from timeseries_generator import Generator, HolidayFactor, SinusoidalFactor, WeekdayFactor, WhiteNoise

class Factory_v2:

    def __init__(self, output_rate, data, 
                 split, capacity, repair_cost, 
                 resupply_cost, storage_cost, 
                 resupply_qty, lead_time, 
                 product_price, file_path):

        self.working = 1
        self.output = 0

        # Cost elements
        self.output_rate = output_rate
        self.capacity = capacity
        self.repair_cost = repair_cost
        self.resupply_cost = resupply_cost
        self.storage_cost = storage_cost
        self.resupply_qty = resupply_qty
        self.lead_time = lead_time
        self.product_price = product_price

        self.repair_counter = 0
        self.repair_time = 0
        self.repair_status = 0
        self.resupply_status = 0

        # List of Lead Time for Countdown
        self.resupply_list = np.array([])

        # Data
        try:
            self.df = load_data(file_path, data, split)
        except:
            print('Data not found. Please run the create_data function.')

        # Demand
        self.demand_dist = Generator(factors = {CountryGdpFactor(),
                                                EUIndustryProductFactor(),
                                                HolidayFactor(holiday_factor = 1.5),
                                                WeekdayFactor(
                                                    factor_values = {4: 1.05, 5: 1.15, 6: 1.15}  
                                                ), # Here we assign a factor of 1.05 to Friday, and 1.15 to Sat/Sun
                                                SinusoidalFactor(wavelength = 365, 
                                                                amplitude = 0.2, 
                                                                phase = 365/4, 
                                                                mean = 1),
                                                WhiteNoise(stdev_factor = 0.3)},
                                    features = {"country": ["Netherlands"],
                                                "store": ["store1"],
                                                "product": ["winter jacket"]},
                                    date_range = pd.date_range(start = '2010', periods = (len(self.df) + 1), freq = 'D'),
                                    base_value = 4).generate()
               
        self.demand_dist[self.demand_dist.value < 0] = 0
        self.demand_dist = round(self.demand_dist['value']).values
        self.fulfilled_orders = 0
    
    # Resource Usage
    def update_inv(self):

        if self.capacity < self.output_rate:
            return

        if self.working and self.repair_status == 0:
            self.capacity -= self.output_rate
            self.output += self.output_rate

    # Deterioration
    def failure_check(self, time_step):

        # Break down
        if self.df.iloc[time_step].Failure:
            self.working = 0
            return

    # Update Lead time 
    def update_lt(self):

        if len(self.resupply_list) > 0:
            self.resupply_list -= 1 # Count down lead time
            # Replenish Stock
            self.capacity += self.resupply_qty * sum(self.resupply_list <= 0)
            self.resupply_list = self.resupply_list[self.resupply_list > 0]

    def repair(self):

        self.repair_time = 1
        self.repair_counter += 1
        self.working = 1
        self.repair_status = 1

    def resupply(self):

        # Send resupply orders
        self.resupply_list = np.append(self.resupply_list, self.lead_time)
        self.resupply_status = 1

In [3]:
import gym
from gym import spaces
import time
import numpy as np
import pandas as pd
# from gym_pm.envs.Objects import Factory_v2
from IPython.display import display, clear_output

class Assemblyv2_Env(gym.Env):
    metadata = {"render.modes": ["console"]}

    def __init__(self, env_config=None, output_rate=7, 
                 data='PdM2', split='Train', capacity=35,
                 repair_cost=30, resupply_cost=3, storage_cost=2, 
                 backlog_cost=10, resupply_qty=28, lead_time=3, 
                 product_price=75, file_path='Gym-PM/gym_pm/data/'):

        # Cost elements
        self.output_rate = output_rate
        self.capacity = capacity
        self.repair_cost = repair_cost
        self.resupply_cost = resupply_cost
        self.storage_cost = storage_cost
        self.backlog_cost = backlog_cost
        self.resupply_qty = resupply_qty
        self.lead_time = lead_time
        self.product_price = product_price

        # Initialize everything
        self.data = data
        self.split = split
        self.file_path = file_path
        self.reset()

        # Episode length
        self.max_duration = len(self.machine.df) # max time
        self.max_resource = self.max_duration * max(output_rate, resupply_qty) + self.machine.demand_dist.sum() # Set to a high value

        # action space
        self.action_space = spaces.Discrete(3)

        # obs space
        obs_bound = load_data(self.file_path, data, 'Bound')
        obs_bound = obs_bound.to_dict(orient='index')
        obs_bound.pop('ttf')

        obs_space = {}
        for i, j in obs_bound.items():
            if i == 'Failure':
                obs_space[i] = spaces.MultiBinary(1)
            else:
                obs_space[i] = spaces.Box(low=j['low'], high=j['high'], shape=(1,), dtype=np.float32)

        obs_space['backlog'] = spaces.Box(low=0., high=self.max_resource, shape=(1,), dtype=np.float32)
        obs_space['output'] = spaces.Box(low=0., high=self.max_resource, shape=(1,), dtype=np.float32)
        obs_space['resources'] = spaces.Box(low=0., high=self.max_resource, shape=(1,), dtype=np.float32)
        obs_space['resupply_queue'] = spaces.Box(low=0., high=self.max_resource, shape=(1,), dtype=np.float32)

        self.observation_space = spaces.Dict(obs_space)

    def reset(self):

        # reset timer
        self.timer = 0
        # reset time_step
        self.time_step = 0        
        # reset backlog
        self.backlog = 0

        self.machine = Factory_v2(output_rate=self.output_rate, 
                                  data=self.data,
                                  split=self.split,
                                  capacity=self.capacity,                 
                                  repair_cost=self.repair_cost, 
                                  resupply_cost=self.resupply_cost,
                                  storage_cost=self.storage_cost, 
                                  resupply_qty=self.resupply_qty,
                                  lead_time=self.lead_time, 
                                  product_price=self.product_price,
                                  file_path=self.file_path)

        return self.observation()

    def observation(self):

        state = self.machine.df.iloc[self.time_step].to_dict()
        state['backlog'] = self.backlog
        state['output'] = self.machine.output
        state['resources'] = self.machine.capacity
        state['resupply_queue'] = len(self.machine.resupply_list)
        state.pop('ttf')
        state = {i: np.array([j], dtype='float32') for (i, j) in state.items()}

        return state

    def get_reward(self):

        reward = 0.
        # Repair Cost
        reward -= self.machine.repair_cost * self.machine.repair_status * self.machine.repair_time
        # Resupply Cost
        reward -= self.machine.resupply_cost * self.machine.resupply_status * self.machine.resupply_qty
        # Inventory Cost
        reward -= self.machine.capacity * self.machine.storage_cost
        reward -= self.machine.output * self.machine.storage_cost
        # Backlog Cost
        reward -= self.backlog * self.backlog_cost
        # Sales Revenue
        reward += self.machine.fulfilled_orders * self.machine.product_price
        if self.machine.working == False:
            reward -= 200

        return reward

    def check_done(self):

        if self.timer >= self.max_duration:
            done = True
        else:
            done = False

        return done

    def fulfil_demand(self):

        self.backlog += self.machine.demand_dist[self.timer]

        if self.machine.output == 0:
            self.machine.fulfilled_orders = 0
        elif self.backlog > self.machine.output:
            self.backlog -= self.machine.output
            self.machine.fulfilled_orders = self.machine.output
            self.machine.output = 0
        else:
            self.machine.output -= self.backlog 
            self.machine.fulfilled_orders = self.backlog
            self.backlog = 0

    def step(self, action):
       
        # Reset Status
        self.machine.repair_status = 0
        self.machine.resupply_status = 0

        # Replenish Stock
        self.machine.update_lt()
        # Deterioriation
        self.machine.failure_check(self.time_step)

        # Interactions (Add more as desired)
        if action == 0:
            self.machine.repair()
            # Reset time_step
            self.time_step = np.random.choice(self.machine.df[self.machine.df.age == 1].index)
        elif action == 1:
            self.machine.resupply()
        else:
            pass

        # Inventory
        self.machine.update_inv()
        # Reduce Backlog
        self.fulfil_demand()

        obs = self.observation()
        reward = self.get_reward()
        done = self.check_done()
        info = {}

        self.timer += 1

        if (self.machine.working) and (self.machine.repair_status == 0):
            self.time_step += 1

        return obs, reward, done, info

    def render(self, mode='console'):

        if (self.machine.working) and (self.machine.repair_status == 0):
            result = self.machine.df.iloc[self.time_step - 1][['age', 'ttf', 'Failure']]
        else:
            result = self.machine.df.iloc[self.time_step][['age', 'ttf', 'Failure']]

        result.Failure = result.Failure.astype(bool)
        result['resources'] = round(self.machine.capacity, 2)
        result['repair_count'] = self.machine.repair_counter
        result['reward'] = self.get_reward()
        result['time_step'] = int(self.time_step)
        result['duration'] = int(self.timer)
        result['lead_time'] = self.machine.resupply_list.copy()
        result['backlog'] = self.backlog
        result['output'] = self.machine.output
        result['resupply_queue'] = len(self.machine.resupply_list)
        result = result.to_frame('Results')
            
        if mode == 'human':

            clear_output(wait=True)
            display(result)
            time.sleep(1)

        return result
            
    def close(self):
        pass

# Test Environment

In [4]:
from stable_baselines3.common.env_checker import check_env

env = Assemblyv2_Env()
check_env(env, warn=True)

In [5]:
env.reset()
for _ in range(5):
    env.render('human')
    env.step(env.action_space.sample()) # take a random action

    if env.check_done():
        break

,Results
age,4.0
ttf,16.0
Failure,False
resources,7
repair_count,0
reward,427.0
time_step,4
duration,4
lead_time,"[1.0, 2.0, 3.0]"
backlog,0
